# Radio Modulation with FINN - Notebook #4 of 5
This notebook walks you through the steps to use FINN to create a bitstream file for the FPGA! 
Specifically, we:
1. Create IP/Verilog per layer (with Vitis HLS and IPI)
2. Create a stitched design (with Vitis HLS and IPI)
3. Create a Vivado project 
4. Run synthesis (to generate a bitfile) 
5. Generate the runtime driver (Python file which hold the model definition and communicate with PYNQ API to run inference on FPGA)

### Cell Overview 
This notebook walks you through steps:
1. Defining custom steps for the builder
2. Setting up Dataflow Builder
3. Build Artifacts
4. Setting Parameters for the Dataflow Architechture.


### FINN Pipeline Map
Throughout these notebooks, you will begin to understand the FINN pipeline! In order the pipeline is:
1. Dataset and Vanilla model
2. Brevitas Model
3. Transforming the Brevitas Model to tidy.onnx
4. **Transforming tidy.onnx to bitstream** (you are here)
5. Loading the bitstream on the FPGA!

## 1. Defining custom steps for the builder
Specifically, we use FINNs _builders_ to create a _dataflow-stype_ architecture. 

Because our model (VGG10) is compatible with the template builder that FINN provide, we do not need to design our own dataflow architechture. We can use their template, with a few minor additional steps to handle our 1D convolutional layers.

A _builder_ can be thought of as a pseudo "compiler toolchain", but for compiling with a FPGA target!

The _builder_ has a few steps already prepared for us. However, since we are using a 1D conv layer, we will need to add 2 more custom steps to convert them from 1D to 2D. FINN works with 4D (NHWC) internally, even with feature maps with only 1 spatial dimension.

`step_pre_streamline` is for converting from our model from 3D tensors to 4D tensors. This is because we initially use 1D convolutional layers. This means the input shape will be changed from `1x2x1024` to `1x2x1024x1`

`step_convert_final_layers` is for converting the final layers (linear and topK) to hardware layers

The code below is from the following finn example: [CustomSteps](https://github.com/Xilinx/finn-examples/blob/main/build/vgg10-radioml/custom_steps.py)

In [1]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron

from qonnx.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
from qonnx.transformation.general import GiveUniqueNodeNames

import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
import finn.transformation.streamline.absorb as absorb
from finn.builder.build_dataflow_config import DataflowBuildConfig
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg

from datetime import datetime
import os
from pathlib import Path 


def step_pre_streamline(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(Change3DTo4DTensors())
    model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
    return model


def step_convert_final_layers(model: ModelWrapper, cfg: DataflowBuildConfig):
    model = model.transform(to_hw.InferChannelwiseLinearLayer())
    model = model.transform(to_hw.InferLabelSelectLayer())
    model = model.transform(GiveUniqueNodeNames())
    return model

## 2. Setting up Dataflow Builder

Define the path to the ONNX model and the target platform

In this example:
- We will use the `finn.onnx` model that has just gone through the `network-surgery` from previous step (`notebook 3/5`)
- The only target platform we will be using for this example is `ZCU104` 

In [2]:
dt=datetime.today().strftime('%Y_%m_%d')
#The output model_name, used for naming the output directory
model_name = 'radio_27ml_finn_2bit'

#include date and random hex code to avoid duplicate file when output 
final_name=model_name+"_"+dt+"_"+os.urandom(3).hex()+"/"
#The onnx model that was previously generated by tutorial 3
model_file ='27ml_rf_2bit(done)/onnx_models/radio_27ml_finn.onnx'

# which platforms to build the networks for
zynq_platform = "ZCU104"

## 3. Build Artifacts 

When FINN is compiling the bitfile, many artifacts are generated. There are three important locations for artifacts:
1. ***Temporary Artifacts:*** saved in "notebooks/tmp". In the below cell we set environment variable FINN_BUILD_DIR to this directory.
2. ***Output Artifacts:*** saved in "notebooks/output". These include the Vivado project files that can be open in Vivado.
3. ***Deploy Artifacts:*** saved in "notebooks/deploy". These are the files to be copied to the FPGA.

**Temporary Artifacts** are _overwritten_ which each new run. These files are created by the FINN compiler. Specifically, in the tmp folder there are files generated for the coressponding HLS, RTL, and IP for layers.

**Output Artifacts** Contain the Vivado project itself that can be open directly in Vivado. In total this contains (1) Bitfile (2) Intermediate Models (3) The stitched IP for the model (4) other configuation and log files. 

**Deploy Artifacts** Contain this the files to be copied to the FPGA. This contains (1) The bitfile (2) The driver python file to be ran on the FPGA. 

**Notice:** The `tmp/` directory is not commited onto our github repository to save space. 

In [3]:
import os
import shutil
import glob
from pathlib import Path

#Define the temporary folders 
temporary_artifacts = Path('tmp')
os.environ["FINN_BUILD_DIR"]=str(temporary_artifacts.absolute())
temporary_artifacts.mkdir(exist_ok=True)

# Define the Output Artifacts directory and create it 
output_dir= Path("output")
output_dir.mkdir(exist_ok=True)

#Remove all intermediate transformations from previous runs 
print(f'Clearing the old temporary artifacts at: {temporary_artifacts}')
shutil.rmtree(temporary_artifacts)

print(f'output will be generated in {output_dir}')

Clearing the old temporary artifacts at: tmp
output will be generated in output


## 4. Setting Parameters for the Dataflow Architechture.

For this example, we define:
1. `target fps`: Target inference performance in frames per second.
2. `clock period`: Target clock period (in nanosecond) for Vivado synthesis.
3. `select_build_steps`: The architechture of our build flow, going from the onnx model to the bitfile that can be run on FPGA.
4. `select_generate_output`: What information about the product we want to see.
    - Documentation on what the generated outputs mean: [Generated Outputs](https://finn.readthedocs.io/en/latest/command_line.html#generated-outputs)

    
Documentation for parameters can be found here: [BuildConfig](https://finn.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.DataflowBuildConfig)



In [4]:
# Target inference performance in frames per second
def select_target_fps(platform):
    return 4500

# Target clock period (in nanoseconds) for Vivado synthesis.
# Frequency (MHz) = 1000 / clock_period_ns 
# e.g. synth_clk_period_ns=5.0 will target a 200 MHz clock.
def select_clk_period(platform):
    return 5.0 

# assemble build flow from custom and pre-existing steps
def select_build_steps(platform):
    return [
        #------------Network-Preparation------
        "step_tidy_up",
        step_pre_streamline, #Custom steps above
        "step_streamline",
        "step_convert_to_hw",
        step_convert_final_layers,  #Custom steps above
        "step_create_dataflow_partition",
        "step_specialize_layers",
        "step_target_fps_parallelization",
        "step_apply_folding_config",
        "step_minimize_bit_width",  
        "step_generate_estimate_reports",
        #------------Hardware-Build-(finn generate instruction files for VITIS HLS)----
        "step_hw_codegen",
        "step_hw_ipgen",
        "step_set_fifo_depths",
        "step_create_stitched_ip",
        #------------HW-synthesis--------------------------
        "step_measure_rtlsim_performance",
        "step_out_of_context_synthesis",
        "step_synthesize_bitfile",
        "step_make_pynq_driver",
        "step_deployment_package",
    ]
    
#What information we want to see.
def select_generate_output(platform):
    return [
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.BITFILE, #This is how we tell the builder to generate the bitfile
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER, 
    ]

## Setup the `start_dataflow` function.
- The input being the `platform_name`. In our example, this would be `ZCU104`
- The function goes through 3 major steps:
    1. Get the `release platform name`, `shell flow type`, and `vitis platform` and create a directory which will store its bitfile.
    2. Set up a config for the builder based on the output from step 1.
    3. Start running the architechture
- The output is the `config file` and the `output directory`


In [5]:
def start_dataflow(platform_name, output_dir):
    '-----------------------Get the platform of the target board--------------------------'
    shell_flow_type = build_cfg.ShellFlowType.VIVADO_ZYNQ
    vitis_platform = None
    release_platform_name = platform_name

    
    '-----------------------Define the config for the build architechture---------------'
    cfg = build_cfg.DataflowBuildConfig(
        steps=select_build_steps(platform_name),
        output_dir=str(output_dir.joinpath(f"output_{final_name}_{release_platform_name}")),
        synth_clk_period_ns=select_clk_period(platform_name),
        target_fps=select_target_fps(platform_name), #Target FPS, not guaranteed the model will achieve
        board=platform_name,
        shell_flow_type=shell_flow_type,
        vitis_platform=vitis_platform,
        split_large_fifos=True,
        standalone_thresholds=True,
        # enable extra performance optimizations (physopt)
        vitis_opt_strategy=build_cfg.VitisOptStrategyCfg.PERFORMANCE_BEST,
        generate_outputs=select_generate_output(platform_name),        
    )
    
    '-----------------------Start the build flow--------------------------------------------'
    # Start the build flow, with the input being the [onnx model] and the [config file]
    build.build_dataflow_cfg(model_file, cfg)
    
    return cfg

## Start running the architechture
We will iterate through all platform assigned in `platforms_to_build`, and run the dataflow architechture

In [6]:
cfg=start_dataflow(zynq_platform, output_dir)

Building dataflow accelerator from 27ml_rf_2bit(done)/onnx_models/radio_27ml_finn.onnx
Intermediate outputs will be generated in /home/phu/repos/RadioFINN/notebooks/tmp
Final outputs will be generated in output/output_radio_27ml_finn_2bit_2025_07_17_c8f176/_ZCU104
Build log is at output/output_radio_27ml_finn_2bit_2025_07_17_c8f176/_ZCU104/build_dataflow.log
Running step: step_tidy_up [1/20]
Running step: step_pre_streamline [2/20]
Running step: step_streamline [3/20]
Running step: step_convert_to_hw [4/20]
Running step: step_convert_final_layers [5/20]
Running step: step_create_dataflow_partition [6/20]
Running step: step_specialize_layers [7/20]
Running step: step_target_fps_parallelization [8/20]
Running step: step_apply_folding_config [9/20]
Running step: step_minimize_bit_width [10/20]
Running step: step_generate_estimate_reports [11/20]
Running step: step_hw_codegen [12/20]
Running step: step_hw_ipgen [13/20]
Running step: step_set_fifo_depths [14/20]
Running step: step_create_st

In [7]:
#Create deploy dir
def create_deploy_dir(cfg, final_name):
    # copy output deploy packages and rename bitfile
    original_deploy_dir = Path(cfg.output_dir).joinpath("deploy") 
    new_deploy_dir= Path("deploy").joinpath(f"{cfg.board}_{final_name}")
    print(f'deploy dir is created at {new_deploy_dir.resolve()}')
    # Define the files we are going to check for from the original deplay 
    # dir, and rename when we move to our deplay directory 
    files_to_check_and_rename = [
        "finn-accel.bit",
        "finn-accel.hwh",
        "finn-accel.xclbin",
    ]
    
    #copy output/[model]/deploy to /deploy
    if original_deploy_dir.exists():
        shutil.copytree(original_deploy_dir,new_deploy_dir, dirs_exist_ok=True)

    shutil.copy("Tutorial5_Load_Bitstream_on_FPGA.ipynb",new_deploy_dir.joinpath("driver"))

    for f in files_to_check_and_rename:
        src_file = new_deploy_dir.joinpath("bitfile").joinpath(f)
        if src_file.is_file():
            shutil.copy(src_file, new_deploy_dir.joinpath("driver").joinpath(f))
            
create_deploy_dir(cfg, final_name)

deploy dir is created at /home/phu/repos/RadioFINN/notebooks/deploy/ZCU104_radio_27ml_finn_2bit_2025_07_17_c8f176


Note: The next tutorial notebook (Tutorial 5) is meant to be run on FPGA. The notebook is automatically copied in the deploy dir

## Now that we have the generated bitfile, we can perform inference on the FPGA.

First we need to zip the generated directory `deploy/[your_model]`. There are many ways to do it. Here we can do it in the terminal:

To copy this notebook from you HOST machine to the FPGA and run ***on the fpga*** (substitute IP with the IP of the FPGA device)

1. `scp -r deploy xilinx@IP:/home/xilinx`
2. `ssh -L 8888:localhost:8888 xilinx@IP`
3. `sudo -E jupyter notebook --port 8888` 
4. Click the URL generated by jupyter notebook to enter jupyter and select this script. 

# About The Report

<font color=orange> **NOTE**: Do not remove all the generated files in `/tmp` yet. We will need them for running implemetation on VIVADO </font>


### FINN Generated Reports
Inside the generated output folder, (eg. `output/[model_name]/_ZCU104/report/`), there will be estimated reports generated by finn.

Estimated performance (**throughput fps, latency in ns, node with highest cycle**, ...) can be found in __estimate_network_performance.json__

### Run Implematation with VIVADO
Aside from generated reports, we can also run implemetation on the generated VIVADO project of the model to get **LUT**, **FF** and **BRAM** utilization

Ensure the final generated VIVADO project can be found in the `output/[model_name]/[platform]/stiched_ip/finn_vivado_stitch_proj.xpr`

We can now open VIVADO, and open a project that has the path pointing at the `stich_proj.xpr` above.

Once the project is opened, we can run synthesis and implementation on VIVADO by clicking `RUN SYNTHESIS` and then `RUN IMPLEMENTATION` on the left panel, which will give us the **Utilization reports**

![VIVADO_Overview](https://raw.githubusercontent.com/UCdasec/RadioFINN/refs/heads/main/ref_images/vivado_proj_overview.jpg)

![Check_Part_ID](https://raw.githubusercontent.com/UCdasec/RadioFINN/refs/heads/main/ref_images/vivado_proj_tutorial_1.jpg)

![Utilization_Report](https://raw.githubusercontent.com/UCdasec/RadioFINN/refs/heads/main/ref_images/vivado_proj_tutorial_2.jpg)